__Data Fitting__

In this tutorial, we use the aggregator to load models and data from a non-linear search and use them to reperform fits to the data.

In [7]:
import autofit as af
import autolens as al
import autolens.plot as aplt

import matplotlib.pyplot as plt

Below, we set up the aggregator as we did in the previous tutorial.

In [8]:
workspace_path = "/home/jammy/PycharmProjects/PyAuto/autolens_workspace/"
output_path = workspace_path + "output"
aggregator_results_path = output_path + "/aggregator_sample_beginner"

af.conf.instance = af.conf.Config(
    config_path=str(workspace_path + "/config"), output_path=str(output_path)
)

aggregator = af.Aggregator(directory=str(aggregator_results_path))


Phases were found in these directories:

/home/jammy/PycharmProjects/PyAuto/autolens_workspace/output/aggregator_sample_beginner/lens_sie__source_sersic__1/pipeline__lens_sie__source_inversion/source__pix_voro_mag__reg_const__mass__with_shear/phase_3__source_inversion/phase_tag__sub_2
/home/jammy/PycharmProjects/PyAuto/autolens_workspace/output/aggregator_sample_beginner/lens_sie__source_sersic__1/pipeline__lens_sie__source_inversion/source__pix_voro_mag__reg_const__mass__with_shear/phase_2__source_inversion_initialization/phase_tag__sub_2
/home/jammy/PycharmProjects/PyAuto/autolens_workspace/output/aggregator_sample_beginner/lens_sie__source_sersic__1/pipeline__lens_sie__source_inversion/source__pix_voro_mag__reg_const__mass__with_shear/phase_1__source_sersic/phase_tag__sub_2
/home/jammy/PycharmProjects/PyAuto/autolens_workspace/output/aggregator_sample_beginner/lens_sie__source_sersic__0/pipeline__lens_sie__source_inversion/source__pix_voro_mag__reg_const__mass__with_shear/phase_3__

Again, we create a list of the MultiNestOutputs of each phase.

In [9]:
pipeline_name = "pipeline__lens_sie__source_inversion"
phase_name = "phase_3__source_inversion"

multi_nest_outputs = aggregator.filter(phase=phase_name).output

We can also use the aggregator to load the dataset of every lens our pipeline fitted. This returns the dataset as the "Imaging" objects we passed to the pipeline when we ran them.

In [6]:
datasets = aggregator.filter(phase=phase_name).dataset
print("Datasets:")
print(datasets, "\n")
print(datasets[0].image)

Datasets:
[<autoarray.dataset.imaging.Imaging object at 0x7fc91c771128>, <autoarray.dataset.imaging.Imaging object at 0x7fc91c771160>, <autoarray.dataset.imaging.Imaging object at 0x7fc91c771ac8>] 

10000


Lets plot each dataset's subplot.

In [ ]:
[aplt.imaging.subplot_imaging(imaging=dataset) for dataset in datasets]

We'll also need the masks we used to fit the lenses, which the aggregator also provides.

In [ ]:
masks = aggregator.filter(phase=phase_name).mask
print("Masks:")
print(masks, "\n")

Lets plot each dataset's again now with its mask.

In [ ]:
[aplt.imaging.subplot_imaging(imaging=dataset, mask=mask) for dataset, mask in zip(datasets, masks)]

To reperform the fit of each most-likely lens model we'll need the masked imaging used by that phase.

In [ ]:
masked_imagings = [
    al.masked.imaging(dataset=dataset, mask=mask)
    for dataset, mask in zip(datasets, masks)
]

Okay, we're good to go! Lets use each most likely instance to create the most-likely tracer, and fit the masked imaging using this tracer for every lens.

In [ ]:
most_likely_model_instances = [
    out.most_probable_model_instance for out in multi_nest_outputs
]

most_likely_tracers = [
    al.Tracer.from_galaxies(galaxies=instance.galaxies)
    for instance in most_likely_model_instances
]

most_likely_fits = [
    al.fit(masked_dataset=masked_imaging, tracer=tracer)
    for masked_imaging, tracer in zip(masked_imagings, most_likely_tracers)
]

[aplt.fit_imaging.subplot_fit_imaging(fit=fit) for fit in most_likely_fits]

The benefit of inspecting fits using the aggregator, rather than the files outputs to the hard-disk, is that we can customize the plots using the PyAutoLens plotters.

In [ ]:
plotter = aplt.Plotter(
    figure=aplt.Figure(figsize=(12, 12)),
    labels=aplt.Labels(title="Custom Image", titlesize=24, ysize=24, xsize=24),
    ticks=aplt.Ticks(ysize=24, xsize=24),
    cmap=aplt.ColorMap(norm="log", norm_max=1.0, norm_min=1.0),
    cb=aplt.ColorBar(ticksize=20),
    units=aplt.Units(in_kpc=True)
)

[aplt.fit_imaging.normalized_residual_map(fit=fit, plotter=plotter) for fit in most_likely_fits]

Making this plot for a paper? You can output it to hard disk.

In [ ]:
plotter = aplt.Plotter(
    output=aplt.Output(
        path=workspace_path + "/output/path/of/file/", filename="publication", format="png"
    ),
)